# Master Notebook

This notebook is the homepage (for the analysis section at least) and should direct you to where you need to go, and guide you through the whole process.

Some of the tasks and steps are pretty intensive, and would make this into the world's longest mega notebook if we did them here, so there are some sub-notebooks you will be directed to.

There are also a number of modules included that we use via import statements. These contain code that I felt complicated the otherwise logical and easy to follow process, so I hid it in these modules. You're welcome to take a look if you are curious and confident, or you can of cource choose to remain blissfully unaware.


### Initialization

Because of the unofficial nature of these modules, we need to tell python where to find them. In this first cell you'll want to replace the path with the path to get to the folder containg your QAOP scripts. 

You're also welcome to put your data files in a seperate location from these notebooks and the code. If you choose to do this, you should specify that location in the cell below.

Run the following cells before continuing; the cells should make sure that everything is prepped and ready.

In [15]:
import sys
#My QAOP folder is in: /Users/aidanmcclung/Desktop/Summer_Exoplanets

codeFilePath = '/Users/aidanmcclung/Desktop/Summer_Exoplanets'  #/QAOP/ <- folder containing QAOP folder.
sys.path.append(codeFilePath) #this lets python know to look here for import statements

We'll need to copy and paste that ^ cell into every new notebook we use, so be ready for that! :)

In [13]:
#We want to save our paths to a file so that our code modules can use it as well!
#In future notebooks we just use a method in the QAOP_Utils module to look up the file.

#This is where you should change the folder you have your data in if you need to
dataFilePath = '/Users/aidanmcclung/Desktop/Summer_Exoplanets/Demo/'
#dataFilePath = ''

import io

with open(codeFilePath+'QAOP/config.txt','w') as config_file:
    config_file.write(f"codeFilePath='{codeFilePath}'")
    config_file.write("\n")
    config_file.write(f"dataFilePath='{dataFilePath}'")

In [12]:
#verify that worked by trying to read it and printing the results
with open(codeFilePath+'QAOP/config.txt','r') as config_file:
    testcodeFilePath = config_file.readline()
    testdataFilePath = config_file.readline()
    print(testcodeFilePath)
    print(testdataFilePath)
#modules in the QAOP package should now try and look for this config file automatically.

codeFilePath='/Users/aidanmcclung/Desktop/Summer_Exoplanets/'

dataFilePath='/Users/aidanmcclung/Desktop/Summer_Exoplanets/Demo/'


Throughout the subsequent notebooks there are a few folders that things will try to call and access. These should be in the dataFilePath you specified, and the following cell will make them if they don't exist

In [19]:
#we need to check and maybe create all the folders we will want to use:
import os
required_directories = ['input','output','myWorking','photometry']

for rq_dir in required_directories:
    if not os.listdir(dataFilePath).count(rq_dir): 
        print(rq_dir,": directory does not exist, creating one")
        os.mkdir(dataFilePath+rq_dir)

# Steps

## Getting The Data

The first thing we have to do is actually get our data. Navigate to data.itelescope.net, and log in. Next find the images for your observation, then download and unzip them. 

From there, we should open up one of the images in DS9 (See OnQ if unsure whet that program is) and check what we have for a peak value on the target.

For this example, I used image 6, and had a peak of 37k.

Next we need to rename the images, which would be a bit tedious to do manually.

## Renaming Script
With this script I get all of my (overly) descriptive file names renamed into '###.fit' in a folder called 'input', which is what we'll need for our next script. There was/is a seperate notebook for this, but it is very simple so I added a few cells to do the same thing here.

When you do this, you'll need to replace the `downloaded_images_path` with the path that will be relevant for you. There are 4 commented lines demonstrating how I navigated from my root project folder to where the images are, and we then move them by 'renaming' them. You can follow a similar process. As well, if you created a seperate area for your data to go, you may need to start off the path with a `dataFilePath+` as shown in the final line

If you leave the prefix blank, like I had, you should have unzipped your downloads from iTelescope to the same folder where this notebook is running from.

If you need to navigate back/up a directory, you can do so by adding `'../'` to the relevant location in your (relative) path.

In [30]:
import os

To re-emphasize:

you'll need to replace the `downloaded_images_path` with the path that will be relevant for you. 

In [27]:
#print(os.listdir())
#print(os.listdir('T21'))
#print(os.listdir('T21/TrES-2'))
#print(os.listdir('T21/TrES-2/20230807'))
downloaded_images_path = 'T21/TrES-2/20230807/'
#downloaded_images_path = dataFilePath + 'T21/TrES-2/20230807/'

After identifying where our images are, we rename them with the following simple loop:

In [28]:
for filename in os.listdir(downloaded_images_path):
    #either of the two following lines are intended to filter out and skip anything thats not an iTelescope image
    if not filename[-4:] == '.fit': continue
    #if not filename[10:] == 'calibrated': continue
    
    #The last seven digits of our images will be ###.fit, so that's what we keep
    os.rename(downloaded_images_path+filename,dataFilePath+"input/"+filename[-7:])

## AstrometryNotebook

For the next major step, you'll need to open up a suplementary notebook: "AstrometryNotebook". You'll also want to make sure you have the "client.py" module installed somewhere. It should be included in the QAOP modules folder, but the subnotebook will have a new python kernal and not know to look there, so you'll have to copy the same path defining cell we had at the start of this notebook.

The Astrometry notebook is quite well detailed! Basically we're adding all of our images to nova.astrometry.net to get wcs data. The basic summary of the process is that the notebook will take every image in the 'input' folder, and then upload them to the site. It will then loop through the list it created of all the files it uploaded, and download the new versions. These just appear in the same directory the script is in, so the last thing it does is move them into the folder 'output'.

*For those that are curious, there are a few cells at the end of the Astrometry notebook that compares and shows what changed when we did this process.*

## StarIDNotebook
The next step of our analysis is to identify the stars, and give them names that we will use in our analysis.

We don't particularly care or need to know their actual names at this point in our analysis, so I generally name them all like 'sA,sB,...'. 

The one difference is that we need to identify which one is the target, so I will often add another identifier. The point of all of this is to have a list of RA and DEC for stars that we will use in our future analysis.

All of this will be done using the "StarIDNotebook" at the moment, although a QAOP module is in the works to do this functionality. For now though, open up the suplementary notebook, and follow it's steps to create the `nameloc.csv` file.

*It is pretty badly documented at the moment, but I am going to prioritize finishing the documentation of all the things and steps in the process before I really go back and explain that whole mess... It is, of all the suplementary notebooks, the last reamining corpse of my original mega notebook, which means it has a lot of confusing extra things and hurts my brain to look at...*

## Radial Profiling Notebook

After we have given names to each of our stars, the final step in preparing for photometry is to know how big of an aperture to use. We'll do this using a relative scale based on the Full Width Half Max of the stars in our images, which we will find with this notebook.

After going through this notebook, you should have created an `apertures.csv` file

*I totally forgot about detailing this notebook as I was writing and explaining in the master, and I had thought I was done and ready to upload some to github, but then realized I forgot it. This was a mental blow, and I shall leave properly explaining for the first update and continue uploading to github*

## Photometry

The next step is perhaps my favourite, and, as of the current moment, the most 'behind the scenes'-y. 

The process does exist in the `codeDemo` notebook, but it is done using the `QAOP_photometry` package, and we can run that from here.

Right now there is no tutorial/guide, so to understand you'll have to look through the module (which is extremely well documented). I do however plan on eventually making a notebook to introduce and guide the process, provided I have the time to do so.

If you use the notebook, you can skip the following code cells.

In [ ]:
import QAOP.QAOP_photometry as QPhot
import numpy as np
#import os #We should have os from above.
from astropy.table import Table

In [ ]:
#photinstance = QPhot.photInstance(apertureFilePath=dataFilePath+'apertures.csv',resultDir=dataFilePath+'photometry',disableConfig=True)
photinstance = QPhot.photInstance() #It should read in the parameters automatically, if not, use the line above instead
#if you want to restart, you'll need to undo a "autosave" that it does with the following line:
photinstance.clearMasterBuffer()

There are two ways we could loop through our files;
 
We can either look in the folder and run for each file thats in there

Or, we can count up to the highest number and look for a file for each of those numbers

In [ ]:
#This first version runs for all of the files in the output folder, but the times will end up out of order because of how
# You could instead run for a number range, like in the second option.

#Now, we need to run this photometry. We can do that by calling the runForFile method with a file path
#for fn in os.listdir(dataFilePath+'output'):
#    if not fn[:-5] == '.fits' : continue #Filter non fits images that may be hididng
#    filepath = dataFilePath+'output/'+fn
#    photinstance.runForFile(filepath) #def runForFile(self,filepath):

In [ ]:
#Now, we need to run this photometry. We can do that by calling the runForFile method with a file path,
#we can get that file path using a number with a helpful little function in the utils module
from QAOP.QAOP_utils import getFilepath
#  This method should automatically use the 'dataFilePath' that we wrote to the config file earlier

#This version counts up numbers; but you'll need to skip any that would cause problems
highest_image_number = 73

for imgnumber in range(1,highest_image_number+1):
    #if imgnumber == {badnumber}: continue #Skip any numbers that we need to (ex, file corrupted etc)
    #if imgnumber == 30: continue #In the Qatar-1 data, img 30 is corrupted
    filepath = getFilepath(imgnumber)
    #print(filepath) # For Troubleshooting
    photinstance.runForFile(filepath) #function is: runForFile(self,filepath)

In [ ]:
#After we have run for all of our files
photinstance.exportMasterAsSimple()
#print(photinstance.resultDir) # if you're having trouble finding the file, this is where it is

## Plotting

Now, with the conclusion of that step, you should have done all the steps you need in order to plot your light curve.
You'll need to do the differential photometry step still as well, but I've been doing that in my plotting notebook.

For an example, I have included my plotting notebook, but you'll need different plots each time, and so I didn't comment or prepare any scripts for that part.


As a basic summary of what you'll need:

You read in the data by using `photData = Table.read('photometry/simple.ecsv)`. From there, you can access the data that we found. There's a column, `time` which has datetime strings that are pretty funky and we can't use to plot, so for your x-axis you'll need to get just the numeric times out of there. There's a function you can import from the utils package to convert them, if you just pass it the data table you have: `times = genTimesFromTable(photData)`

Next, you can get the fluxes by simply referencing the data with the name of the star you want, for example, if I wanted the y-data to plot flux vs time for my target, I would get it from `photData['target'].data`. Because the data is stored in an astropy table, just using `photData['target']` would give us a subtable with the header type info, which we can probably still plot, but just to be safe I like to extract an array of the values with the `.data` property.

With all that said, I could then plot the data in the way that I am most familiar with.

To do the differential part, we can use either (or both) of two ways; relative flux, and magnitudes.

If you would like to compute the magnitude difference between two stars, there is a function in the utils package `diffMag(flux1,flux2)` that will give you that.  For the relative flux, simply divide the flux of the star you want (probably the target) by the flux of your other comparison star. if you have numpy arrays (or astropy tables), it will handle the element-wise stuff automatically and you can just have `relative_flux = flux1/flux2` in your code.